## Librerias


In [109]:
#Manejo de datos
import pandas as pd
import numpy as np


#Fuzz
!pip install thefuzz
from thefuzz import fuzz

#Uso de strings
import re
!pip install Unidecode
import unidecode  # Para remover tildes

#Google colab
from google.colab import auth
from google.cloud import bigquery

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Base

In [110]:
df1 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base2.csv')
cities_df = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/cities.csv', usecols= ['city']) #Ciudades
df1["campaign_o"] = df1["campaign"] #Para al final en dummy event incluir la campaña original conservando stopwords

### **Df 1**

- customer: Nombre del comercio (eticket/tuboleta)
- campaign: Nombre del evento enviada por web service
- campaign_year: Año de la campaña

### **Df 2**
- customer: Nombre del comercio (eticket/tuboleta)
- slug: 	slug de las campañas.
- campaign: Nombre del evento creada por Armatuvaca.
- campaign_year: Año de la campaña
- event_city: Ciudad de la campaña


_________________________________________________________________________________________________________________________________________________________________


# **Analisis inicial**

Hasta el momento se han creado 359 eventos que aún no están diferenciados por campaña.

In [111]:
len(df1)

359

Exiten 137 campañas de tuboleta y 222 campañas de eticket

In [112]:
df1.groupby(['customer']).count()

,campaign,campaign_year,campaign_o
customer,,,
Tuboleta,137,137,137
eticket,222,222,222


Hay 264 eventos dummy creados internamente, son relativamente menos a la info. enviada por las tiqueteras pues en ocasiones los eventos en eticket/tuboleta que tienen varios días se crean como eventos independientes, ejemplo: Festival Estereo Picnic sábado, Festival Estereo Picnic domingo; internamente se crean como un solo evento (Festival Estereo Picnic)

Además, en ocasiones internamente no se crea el dummy al evento por razones como:



*   Evento con muy pocas ventas
*   Evento muy próximo a realizarse



In [113]:
len(df2)

264

La manera de identificar que un evento es eticket o tuboleta es a través de 'dummy_redirect' sin embargo 4 eventos no tienen un link que redirige al check out **problema**

In [114]:
df2.groupby(['customer'])['campaign_slug'].count().reset_index()

,customer,campaign_slug
0,Tuboleta,116
1,eticket,144


In [115]:
df2.groupby(['customer', 'campaign'])['campaign_slug'].count().reset_index()

,customer,campaign,campaign_slug
0,Tuboleta,&ME from KEINEMUSIK,1
1,Tuboleta,14 Cañonazos Fest,1
2,Tuboleta,ACE,1
3,Tuboleta,ANOTR y Seth Troxler,1
4,Tuboleta,Air Supply,1
...,...,...,...
216,eticket,WOS,1
217,eticket,Wild Brunch Cartagena,1
218,eticket,X BY ADRIATIQUE,1
219,eticket,YSY A,1


Teneindo en cuenta lo anterior, no es una relación uno a uno sino varios registros web service pueden tener un solo registro en 'piggys_piggycampaign'

# **Casos más comunes**



## 1.   **Muchas campañas un solo evento**
* BAUM FESTIVAL 2024  25 DE MAYO  SÁBADO INDIVIDUAL
* BAUM FESTIVAL 2024  25 DE MAYO SÁBADO INDIVIDUAL PROMOTORES
* BAUM FESTIVAL 2024  PROMOTORES
* BAUM FESTIVAL 2024 COMBO 2 DIAS
* BAUM FESTIVAL 2024 COMBO 2 DÍAS  PROMOTORES

Todos los eventos pertenecen a: **'Baum Festival'**, 'baum-festival-2024'
## 2.   **Similitud alta entre varios eventos siendo campañas independientes**

* CAMILO NUESTRO LUGAR FELIZ TOUR **-** Camilo **-** camilo-bogota-2024
* CAMILO NUESTRO LUGAR FELIZ TOUR  2DA FECHA **-** Camilo**-**camilo-bogota-2024-2

## 3. **Campañas que no fueron creadas como dummys**

JHONNY RIVERA  20 AÑOS

## 4. **Campañas para diferentes ciudades sin la etiqueta de la ciudad (Ej: Bogotá)**
* VIVE LA SALSA   PALCOS & GRADERIA - vive-la-salsa-bogota-2024 - Vive La Salsa
* VIVE LA SALSA  (COMBO 2 DÍAS) PALCOS & GRADERIA - vive-la-salsa-bogota-2024 - Vive La Salsa
* VIVE LA SALSA  CALI - vive-la-salsa-cali-2024 - Vive La Salsa
* VIVE LA SALSA  MEDELLÍN - vive-la-salsa-medellin-2024 - Vive La Salsa

## 5. **Campañas sin ningún tipo de coincidencia o bien muy poca**

* TOMORROWLAND PRESENTS CORE  2025  SINGLE - CORE - core-medellin-2025

## 6. **Campañas dummy no clasificadas como eticket/tuboleta**

**Problema extracción de la info.(dummy_redirect)**

* CORE
* Corasound Fest
* Festival Estéreo Picnic 2023
* La Solar

## 7. **Uso de Stopwords**

- "Promotores"
- DTO (Descuento)


# spacy







## **Distancias de Levenshtein**

In [95]:
print(fuzz.ratio('hola', 'hoLA,')) #calcula la distancia de edición basándose en el ORDEN de ambas cadenas de entrada.
print(fuzz.partial_ratio('hola', 'hoLA,'))  #calcula la similitud tomando la cadena más corta y luego la compara con las subcadenas de la misma longitud de la cadena más larga, que se almacena en nombre_completo.
print(fuzz.token_sort_ratio('hola', 'HOLA,')) # Tiene en cuenta las cadenas similares que no están en el orden expresado anteriormente.
print(fuzz.token_set_ratio('hola', 'HOLA,')) # elimina los tokens comunes antes de calcular lo similares que son las cadenas: esto es extremadamente útil cuando las cadenas tienen una longitud significativamente diferente.
print(fuzz.partial_token_sort_ratio('hola', 'HOLA,'))# I guess, toma la frase más corta y no tiene en cuenta el orden de las palabras

44
67
100
100
100


# Metodología

In [ ]:
#Distanciasmás utilizadas
##Ratio = fuzz.ratio(d1.lower(),d2.lower())
##Partial_Ratio = fuzz.partial_ratio(d1.lower(),d2.lower())
##Token_Sort_Ratio = fuzz.token_sort_ratio(d1.lower(),d2.lower())
##Token_Set_Ratio = fuzz.token_set_ratio(d1.lower(),d2.lower())

In [124]:
stopwords = {"promotores", "dto", "fan days", "combo", "festival"}


def remove_stopwords(text):
    """
    Solo remueve stopwords sin modificar mayúsculas/minúsculas en el texto final.
    """
    words = str(text).split()
    filtered_words = [word for word in words if word.lower() not in stopwords]  # Convertimos a minúsculas solo para la comparación
    return " ".join(filtered_words)


In [125]:

def normalize_text(text):
    """
    Normaliza el texto: convierte a minúsculas y remueve tildes
    """
    text = str(text).lower()
    text = unidecode.unidecode(text)
    return text


In [126]:
def load_data():
    """
    Carga los archivos de datos y prepara las ciudades
    """
    print("Leyendo archivos...")
    df1 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base1.csv')
    df2 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base2.csv')
    df1["campaign_o"] = df1["campaign"] #Para al final en dummy event incluir la campaña original conservando stopwords
    cities_df = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/cities.csv')

    df1['campaign'] = df1['campaign'].apply(remove_stopwords)
    df2['campaign'] = df2['campaign'].apply(remove_stopwords)

    for text in df1['campaign'].head():
        print(f"Normalizado: {text}")

    # Normalizar ciudades (sin eliminar stopwords para mantener nombres de ciudades intactos)
    cities = cities_df['city'].apply(lambda x: str(x).lower()).apply(unidecode.unidecode).unique()
    cities = sorted(cities, key=len, reverse=True)

    # Crear diccionario de ciudades normalizadas a originales
    cities_dict = dict(zip(cities_df['city'].apply(lambda x: str(x).lower()).apply(unidecode.unidecode),
                          cities_df['city']))

    return df1, df2, cities, cities_dict

def find_city_in_campaign(campaign_text, cities):
    """
    Busca si alguna ciudad de la lista está en el nombre de la campaña
    """
    # Para ciudades, solo aplicamos lowercase y unidecode, sin eliminar stopwords
    campaign_text = str(campaign_text).lower()
    campaign_text = unidecode.unidecode(campaign_text)

    for city in cities:
        if city in campaign_text:
            return city
    return None


In [127]:
def calculate_match_score(row1, row2, cities, cities_dict):
    """
    Calcula el score de match considerando el texto y la ciudad si existe
    """
    # Buscar ciudad en campaign de base1
    city_in_campaign = find_city_in_campaign(row1['campaign'], cities)

    # Si hay ciudad en el campaign, verificar que coincida con event_city de base2
    if city_in_campaign is not None:
        if 'event_city' in row2:
            event_city_normalized = str(row2['event_city']).lower()
            event_city_normalized = unidecode.unidecode(event_city_normalized)
            if event_city_normalized != city_in_campaign:
                return 0

    # Normalizar textos incluyendo eliminación de stopwords para la comparación
    campaign1 = normalize_text(row1['campaign'])
    campaign2 = normalize_text(row2['campaign'])

    # Calcular scores
    ratio = fuzz.ratio(campaign1, campaign2)
    partial_ratio = fuzz.partial_ratio(campaign1, campaign2)
    token_sort = fuzz.token_sort_ratio(campaign1, campaign2)
    token_set = fuzz.token_set_ratio(campaign1, campaign2)
    partial_token_sort_ratio = fuzz.partial_token_sort_ratio(campaign1, campaign2)

    return (token_sort + token_set + partial_ratio) / 3

In [136]:
def find_best_matches(df1, df2, cities, cities_dict):
    """
    Encuentra las mejores coincidencias considerando las ciudades cuando existan
    """
    all_possible_matches = []
    total_customers = len(df1['customer'].unique())
    print(f"\nProcesando {total_customers} customers...")

    for i, customer in enumerate(df1['customer'].unique(), 1):
        print(f"Procesando customer {i}: {customer}")

        df1_customer = df1[df1['customer'] == customer]
        df2_customer = df2[df2['customer'] == customer]

        for idx1, row1 in df1_customer.iterrows():
            # Buscar ciudad en el nombre de la campaña
            city_in_campaign = find_city_in_campaign(row1['campaign'], cities)

            # Filtrar df2 por año
            df2_year = df2_customer[df2_customer['campaign_year'] == row1['campaign_year']]
            if len(df2_year) == 0:
                df2_year = df2_customer

            for idx2, row2 in df2_year.iterrows():
                # Si hay ciudad en campaign, verificar coincidencia con event_city
                if city_in_campaign is not None:
                    event_city_normalized = normalize_text(row2['event_city']) if 'event_city' in row2 else ''
                    if event_city_normalized != city_in_campaign:
                        continue  # Saltar si las ciudades no coinciden

                match_score = calculate_match_score(row1, row2, cities, cities_dict)

                if match_score > 0:  # Solo agregar si hay posibilidad de match
                    all_possible_matches.append({
                        'customer': customer,
                        'campaign_year': row1['campaign_year'],
                        'campaign_df1': row1['campaign'],
                        'campaign_o_df1': row1['campaign_o'],
                        'campaign_base1_id': idx1,
                        'campaign_df2': row2['campaign'],
                        'campaign_base2_id': idx2,
                        'campaign_slug': row2['campaign_slug'],
                        'city_in_campaign': cities_dict.get(city_in_campaign, city_in_campaign) if city_in_campaign else None,
                        'event_city': row2['event_city'] if 'event_city' in row2 else None,
                        'match_score': match_score
                    })

    if not all_possible_matches:
        raise ValueError("No se encontraron matches posibles entre las bases")

    # Convertir a DataFrame y ordenar
    all_matches_df = pd.DataFrame(all_possible_matches)
    all_matches_df = all_matches_df.sort_values('match_score', ascending=False)

    # Seleccionar mejores matches únicos
    used_base1 = set()
    #used_base2 = set()
    best_unique_matches = []

    for _, match in all_matches_df.iterrows():
        base1_id = match['campaign_base1_id']
        #base2_id = match['campaign_base2_id']

        if base1_id not in used_base1: #and base2_id not in used_base2:
            best_unique_matches.append(match)
            used_base1.add(base1_id)
            #used_base2.add(base2_id)

    results_df = pd.DataFrame(best_unique_matches)
    results_df = results_df.drop(['campaign_base1_id', 'campaign_base2_id'], axis=1)
    results_df = results_df.sort_values(['customer', 'match_score'], ascending=[True, False])

    return results_df

In [137]:
# Cargar datos
df1, df2, cities, cities_dict = load_data()

# Ejecutar el matching
print("Iniciando proceso de matching...")
matches = find_best_matches(df1, df2, cities, cities_dict)

# Filtrar coincidencias
good_matches = matches[matches['match_score'] >= 70]
bad_matches = matches[matches['match_score'] < 70]


# Imprimir resumen
print("\nResumen de resultados:")
print(f"Total de matches únicos: {len(matches)}")
print(f"Matches buenos (score >= 70): {len(good_matches)}")
print(f"Matches malos (< 50): {len(bad_matches)}")

Leyendo archivos...
Normalizado: DUPLAT MOSAICO PIRATA
Normalizado: ROJUU
Normalizado: PIPE BUENO 3ER ROUND
Normalizado: CAMILO NUESTRO LUGAR FELIZ TOUR 2DA FECHA
Normalizado: VIVE LA SALSA CALI
Iniciando proceso de matching...

Procesando 2 customers...
Procesando customer 1: eticket
Procesando customer 2: Tuboleta

Resumen de resultados:
Total de matches únicos: 357
Matches buenos (score >= 70): 285
Matches malos (< 50): 72


In [138]:
matches

,customer,campaign_year,campaign_df1,campaign_o_df1,campaign_df2,campaign_slug,city_in_campaign,event_city,match_score
19336,Tuboleta,2025,MILO J,MILO J,MILO J,milo-j-bogota-2025,None,Bogotá,100.000000
24018,Tuboleta,2025,TIËSTO,TIËSTO,Tiesto,tiesto-bogota-2025,None,Bogotá,100.000000
19172,Tuboleta,2024,PIMPINELA,PIMPINELA,Pimpinela,pimpinela-bogota-2024,None,Bogotá,100.000000
24074,Tuboleta,2024,DRUMCODE,DRUMCODE,DRUMCODE,drumcode-medellin-2024,None,Medellín,100.000000
19145,Tuboleta,2024,CAPITAL FEST,CAPITAL FEST,CAPITAL FEST,capital-fest-bogota-2024,None,Bogotá,100.000000
...,...,...,...,...,...,...,...,...,...
6843,eticket,2023,MAJESTUOSOS 2023 28 DE DICIEMBRE,MAJESTUOSOS 2023 28 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
16002,eticket,2023,MAJESTUOSOS 2023 30 DE DICIEMBRE,MAJESTUOSOS 2023 30 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
9754,eticket,2023,MAJESTUOSOS 2023 13 DE DICIEMBRE,MAJESTUOSOS 2023 13 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
16844,eticket,2023,MAJESTUOSOS 2023 06 DE ENERO,MAJESTUOSOS 2023 06 DE ENERO,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,20.000000


### Clasificados

In [139]:
good_matches

,customer,campaign_year,campaign_df1,campaign_o_df1,campaign_df2,campaign_slug,city_in_campaign,event_city,match_score
19336,Tuboleta,2025,MILO J,MILO J,MILO J,milo-j-bogota-2025,None,Bogotá,100.000000
24018,Tuboleta,2025,TIËSTO,TIËSTO,Tiesto,tiesto-bogota-2025,None,Bogotá,100.000000
19172,Tuboleta,2024,PIMPINELA,PIMPINELA,Pimpinela,pimpinela-bogota-2024,None,Bogotá,100.000000
24074,Tuboleta,2024,DRUMCODE,DRUMCODE,DRUMCODE,drumcode-medellin-2024,None,Medellín,100.000000
19145,Tuboleta,2024,CAPITAL FEST,CAPITAL FEST,CAPITAL FEST,capital-fest-bogota-2024,None,Bogotá,100.000000
...,...,...,...,...,...,...,...,...,...
4748,eticket,2024,BAUM 2024 24 DE MAYO VIERNES INDIVIDUAL,BAUM FESTIVAL 2024 24 DE MAYO VIERNES INDIVI...,Baum,baum-festival-2024,None,Bogotá,73.000000
8562,eticket,2024,BAUM 2024 25 DE MAYO SÁBADO INDIVIDUAL,BAUM FESTIVAL 2024 25 DE MAYO SÁBADO INDIVIDUAL,Baum,baum-festival-2024,None,Bogotá,73.000000
10395,eticket,2024,BAUM 2024 25 DE MAYO SÁBADO INDIVIDUAL,BAUM FESTIVAL 2024 25 DE MAYO SÁBADO INDIVIDU...,Baum,baum-festival-2024,None,Bogotá,73.000000
2042,eticket,2024,FEID EN BOGOTA,FEID EN BOGOTA,Gonzalo Ávila en Bogotá,gonzalo-avila-bogota-2024,Bogotá,Bogotá,71.666667


274 clasificados
- **Bacilos**
- **Andres Calamaro** no se realizo la clasifiación por ciudad
- **Los tigres del norte**
- **THE JUANPIS LIVE SHOW: LOS NOTO ASUSTADOS TEMPORADA 3**	- The Juanpis Live Show: Los Noto Asustados Temporada 2
- **VIVE LA SALSA (COMBO 2 DÍAS) PALCOS & GRADERIA**	Vive La Salsa	vive-la-salsa-medellin-2024
- Camilo segunda fecha
- **FESTIVAL X BY ADRIATIQUE** no tiene match con la campaña correcta debido a que el evento es en 2025 y la fecha límite fue en el 2024, solución alternativa para nuevos eventos utilizar el campo **event_date** de piggys_piggy



### No clasificados

In [140]:
bad_matches

,customer,campaign_year,campaign_df1,campaign_o_df1,campaign_df2,campaign_slug,city_in_campaign,event_city,match_score
22185,Tuboleta,2023,PRIMAVERA PASS VIP,PRIMAVERA PASS VIP,Primavera Sound 2023,primavera-sound-bogota-2023,None,Bogotá,67.000000
20286,Tuboleta,2023,PRIMAVERA PASS GENERAL,PRIMAVERA PASS GENERAL,The Cure Primavera,the-cure-primavera,None,Bogotá,63.000000
24110,Tuboleta,2024,FELIPE PELÁEZ UN SUEÑO LLAMADO SINFÓNICO BOGOTÁ,FELIPE PELÁEZ UN SUEÑO LLAMADO SINFÓNICO BOGOTÁ,Miguel Mateos Sinfónico,miguel-mateos-bogota-2024,Bogotá,Bogotá,57.333333
22073,Tuboleta,2024,ANIME 2DA FECHA,ANIME 2DA FECHA,Anime: Concierto Otaku,anime-concierto-otaku-bogota-2024,None,Bogotá,51.666667
23222,Tuboleta,2025,MAS CERCA DE TI TOUR 2DA FECHA,MAS CERCA DE TI TOUR 2DA FECHA,SÚPER CONCIERTO FERIA DE MANIZALES,super-concierto-feria-de-manizales-2025,None,Manizales,50.333333
...,...,...,...,...,...,...,...,...,...
6843,eticket,2023,MAJESTUOSOS 2023 28 DE DICIEMBRE,MAJESTUOSOS 2023 28 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
16002,eticket,2023,MAJESTUOSOS 2023 30 DE DICIEMBRE,MAJESTUOSOS 2023 30 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
9754,eticket,2023,MAJESTUOSOS 2023 13 DE DICIEMBRE,MAJESTUOSOS 2023 13 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
16844,eticket,2023,MAJESTUOSOS 2023 06 DE ENERO,MAJESTUOSOS 2023 06 DE ENERO,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,20.000000


### Dummy event

In [141]:
matches.loc[matches['match_score'] <= 70, ['match_score', 'campaign_df2', 'campaign_slug']] = None
matches= matches[['campaign_o_df1', 'campaign_slug', 'match_score']]
matches

,campaign_o_df1,campaign_slug,match_score
19336,MILO J,milo-j-bogota-2025,100.0
24018,TIËSTO,tiesto-bogota-2025,100.0
19172,PIMPINELA,pimpinela-bogota-2024,100.0
24074,DRUMCODE,drumcode-medellin-2024,100.0
19145,CAPITAL FEST,capital-fest-bogota-2024,100.0
...,...,...,...
6843,MAJESTUOSOS 2023 28 DE DICIEMBRE,None,NaN
16002,MAJESTUOSOS 2023 30 DE DICIEMBRE,None,NaN
9754,MAJESTUOSOS 2023 13 DE DICIEMBRE,None,NaN
16844,MAJESTUOSOS 2023 06 DE ENERO,None,NaN


# spacy

In [91]:
!pip setuptools wheel
!pip install spacy
import spacy
!python -m spacy download es_core_news_sm

nlp = spacy.load("es_core_news_sm")  # textos en españoool


ERROR: You must give at least one requirement to install (see "pip help install")
ERROR: unknown command "setuptools"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 22.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Leyendo archivos...
Iniciando proceso de matching...
Total de matches: 25347


In [93]:
nlp.Defaults.stop_words

{'a',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'afirmó',
 'agregó',
 'ahi',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún',
 'alli',
 'allí',
 'alrededor',
 'ambos',
 'ante',
 'anterior',
 'antes',
 'apenas',
 'aproximadamente',
 'aquel',
 'aquella',
 'aquellas',
 'aquello',
 'aquellos',
 'aqui',
 'aquél',
 'aquélla',
 'aquéllas',
 'aquéllos',
 'aquí',
 'arriba',
 'aseguró',
 'asi',
 'así',
 'atras',
 'aun',
 'aunque',
 'añadió',
 'aún',
 'bajo',
 'bastante',
 'bien',
 'breve',
 'buen',
 'buena',
 'buenas',
 'bueno',
 'buenos',
 'cada',
 'casi',
 'cierta',
 'ciertas',
 'cierto',
 'ciertos',
 'cinco',
 'claro',
 'comentó',
 'como',
 'con',
 'conmigo',
 'conocer',
 'conseguimos',
 'conseguir',
 'considera',
 'consideró',
 'consigo',
 'consigue',
 'consiguen',
 'consigues',
 'contigo',
 'contra',
 'creo',
 'cual',
 'cuales',
 'cualquier',
 'cuando',
 'cuanta',
 'cuantas',
 'cuanto',
 'cuantos',
 'cuatro',
 'cuenta',
 'cuál',
 'cuáles',
 'cu